## Importing Necessary Libraries

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from local_utils import detect_lp
from os.path import splitext,basename
from keras.models import model_from_json
import pytesseract
import glob
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
wpod_net_path = "wpod-net.json"
from keras.preprocessing.image import ImageDataGenerator, load_img

## Loading Model

In [2]:
def load_model(path):
    try:
        path = splitext(path)[0]
        with open('%s.json' % path, 'r') as json_file:
            model_json = json_file.read()
        model = model_from_json(model_json, custom_objects={})
        model.load_weights('%s.h5' % path)
        print("Loading model successfully...")
        return model
    except Exception as e:
        print(e)

In [3]:
wpod_net = load_model(wpod_net_path)

Loading model successfully...


## Importing Anvil Server

In [4]:
import anvil.server

anvil.server.connect("S44CFBK7DAVUJDB6232MJQYF-RZNK4SZD7CYXKASZ")


Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


## Preprocessing Image from Server and Detecting Text with Pytesseract

In [6]:
import anvil.media
@anvil.server.callable
def imagetotext(file):
    with anvil.media.TempFile(file) as filename:
        img = load_img(filename)
    image = np.array(img)
    img = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    img = img / 255
    img = cv2.resize(img, (224,224))
    Dmax=608
    Dmin=256
    ratio = float(max(img.shape[:2])) / min(img.shape[:2])
    side = int(ratio * Dmin)
    bound_dim = min(side, Dmax)
    _ , LpImg, _, cor = detect_lp(wpod_net, img, bound_dim, lp_threshold=0.5)
    image1 = LpImg[0]
    img_n = cv2.normalize(src=image1, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    img_n = cv2.cvtColor(img_n, cv2.COLOR_BGR2RGB)
    text = pytesseract.image_to_string(img_n)        #obtaining text from pytesseract
    return (text)